<a href="https://colab.research.google.com/github/bardiaHSZD/TrendPrediction/blob/main/Dynamic_Mode_Decomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Environment Installation**

In [ ]:
!apt install octave
!apt-get update --fix-missing
!apt install octave

# **Dynamic Mode Decomposition:** Test

In [6]:
%%writefile dmd_test_01.m

clear all;

%Quick Test of DMD with control.

%Data collection

A = [1.5 0;0 0.1];
x0 = [4;7];
K = [-1];
m = 20;
DataX = x0;
DataU = [0];

B = [1;0];

for j = 1 : m 
   
  DataX(:,j+1) = A * (DataX(:,j)) + B.*(K*DataX(:,j));
  DataU(:,j) = K .* DataX(1,j);
end

%Data matrices
X   = DataX(:,1:end-1);
Xp  = DataX(:,2:end);
Ups = DataU;

%SVD
[U,Sig,V] = svd(X,'econ');

thresh = 1e-10;
r = length(find(diag(Sig)>thresh));

U    = U(:,1:r);
Sig  = Sig(1:r,1:r);
V    = V(:,1:r);

%DMD

A_DMD  = Xp*V*inv(Sig)*U'

%DMDc - B is known 

A_DMDc = (Xp - B*Ups)*V*inv(Sig)*U'

% Extra
[W,D] = eig(A_DMDc)

Phi = Xp * V * inv(Sig) * U'*U * W

% DMD Spectra
t = linspace (0,m,m);
dt = t(2) - t(1);
lambda = diag(D);
omega = log(lambda)/dt;

% Compute DMD Solution
x0 = [4;7];
K = [-1];
m = 20;
DataXdmd = x0;
DataUdmd = [0];

B = [1.0;0.0];

for j = 1 : m   
  DataXdmd(:,j+1) = A_DMDc * (DataXdmd(:,j)) + B.*(K*DataXdmd(:,j));
  DataUdmd(:,j) = K .* DataXdmd(1,j);
end

%Data matrices
XDMD   = DataXdmd(:,1:end-1)
Ups = DataUdmd;

Overwriting dmd_test_01.m


In [7]:
!octave dmd_test_01.m # -W : no window system

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
A_DMD =

   5.0000e-01  -2.1751e-17
  -7.4096e-17   1.0000e-01

A_DMDc =

   1.5000e+00   2.2711e-16
  -7.4096e-17   1.0000e-01

W =

   1.00000  -0.00000
   0.00000   1.00000

D =

Diagonal Matrix

   1.50000         0
         0   0.10000

Phi =

  -0.266675  -0.422947
  -0.084589   0.053335

XDMD =

 Columns 1 through 6:

   4.0000e+00   2.0000e+00   1.0000e+00   5.0000e-01   2.5000e-01   1.2500e-01
   7.0000e+00   7.0000e-01   7.0000e-02   7.0000e-03   7.0000e-04   7.0000e-05

 Columns 7 through 12:

   6.2500e-02   3.1250e-02   1.5625e-02   7.8125e-03   3.9063e-03   1.9531e-03
   7.0000e-06   7.0000e-07   7.0000e-08   7.0000e-09   7.0000e-10   7.0000e-11

 Columns 13 through 18:

   9.7656e-04   4.8828e-04   2.4414e-04   1.2207e-04   6.1035e-05   3.0518e-05
   7.0000e-12   7.0000e-13   7.0000e-14   7.0000e-15   6.9999e-16   6.9994e-17

 Columns 19 and 20:

   1.5259e-05   7.6294e-06
   6.9972e-18   6.9

###**Random Noise**

In [11]:
%%writefile spatio-test.m
clear all, close all, clc

dt = .01;       % time step of signal
L = 10;
t = 0:dt:L;

% signal: sum of two sines at 13Hz and 7Hz
xclean = (14*sin(7*2*pi*t)+5*sin(13*2*pi*t));
% add noise to signal
x = xclean + 10*randn(size(xclean));
ylim([-40 40])
plot(t,x,'Color',[.7 .3 .3],'LineWidth',.9), hold on
plot(t,xclean,'k','LineWidth',1.5)
legend('Noisy signal','Clean Signal')
grid on
set(gcf,'Position',[100 100 500 200])
set(gcf,'PaperPositionMode','auto')
% print('-depsc2', '-loose', '../figures/FFTDMD_signal');
%%
figure
xhat = fft(x);
N = length(t);  % number of samples
xpower = abs(xhat(1:N/2+1))*2/N;
Fs = 1/dt;      % sampling frequency
freqs = Fs*(0:(N/2))/N;
plot(freqs,xpower,'k','LineWidth',1.2)
grid on, hold on

%%
% better de-noising with larger kshift
s = 500;  % number of times to shift-stack signal
for k = 1:s
    X(k,:) = x(k:end-s+k);
end
[U,S,V] = svd(X(:,1:end-1),'econ');

% keep 50 modes and compute DMD spectrum Lambda
r = 50;
Atilde = U(:,1:r)'*X(:,2:end)*V(:,1:r)*inv(S(1:r,1:r));
[W,Lambda] = eig(Atilde);
% convert eigenvalues to continuous time
DMDfreqs = log(diag(Lambda))/dt/2/pi;  
Phi = X(:,2:end)*V(:,1:r)*inv(S(1:r,1:r))*W;

% mode amplitude  (based on first snapshot)
b = Phi\X(:,1);  
%  need to scale power by 2/sqrt(s)
DMDpower = abs(b)*2/sqrt(s) 

scatter(abs(imag(DMDfreqs)),DMDpower,'r')
legend('FFT','DMD')

set(gcf,'Position',[100 100 500 200])
set(gcf,'PaperPositionMode','auto')
% print('-depsc2', '-loose', '../figures/FFTDMD_spectrum');


Overwriting spatio-test.m


In [13]:
!octave spatio-test.m # -W : no window system

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
                                                                               
                  SVD: low rank property (rank = 2, two modes)                 
                                                                               
 0.7 |---------------------------------------------------------------------|   
     |                 +                +                 +                |   
     |                                                                     |   
 0.6 |-+                                                                 +-|   
     |                                                                     |   
 0.5 |-+                                                                 +-|   
     |                                                                     |   
     |                                                                     |   
 0.4 |F+                                